In [1]:
import numpy as np
import pandas as pd

In [93]:
questions = pd.read_csv('data/Questions.csv', encoding='latin1')
questions.head()

,Id,OwnerUserId,CreationDate,Score,Title,Body
0,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...
1,502,147.0,2008-08-02T17:01:58Z,27,Get a preview JPEG of a PDF on Windows?,<p>I have a cross-platform (Python) applicatio...
2,535,154.0,2008-08-02T18:43:54Z,40,Continuous Integration System for a Python Cod...,<p>I'm starting work on a hobby project with a...
3,594,116.0,2008-08-03T01:15:08Z,25,cx_Oracle: How do I iterate over a result set?,<p>There are several ways to iterate over a re...
4,683,199.0,2008-08-03T13:19:16Z,28,Using 'in' to match an attribute of Python obj...,<p>I don't remember whether I was dreaming or ...


In [ ]:
- Clean Body
- map answers(ParentId) with questions(Id)
- add tag

,Id,OwnerUserId,CreationDate,Score,Title,Body
0,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...
1,502,147.0,2008-08-02T17:01:58Z,27,Get a preview JPEG of a PDF on Windows?,<p>I have a cross-platform (Python) applicatio...
2,535,154.0,2008-08-02T18:43:54Z,40,Continuous Integration System for a Python Cod...,<p>I'm starting work on a hobby project with a...
3,594,116.0,2008-08-03T01:15:08Z,25,cx_Oracle: How do I iterate over a result set?,<p>There are several ways to iterate over a re...
4,683,199.0,2008-08-03T13:19:16Z,28,Using 'in' to match an attribute of Python obj...,<p>I don't remember whether I was dreaming or ...


In [12]:
answers = pd.read_csv('data/Answers.csv', encoding='latin1')
answers.head()

,Id,OwnerUserId,CreationDate,ParentId,Score,Body
0,497,50.0,2008-08-02T16:56:53Z,469,4,<p>open up a terminal (Applications-&gt;Utilit...
1,518,153.0,2008-08-02T17:42:28Z,469,2,<p>I haven't been able to find anything that d...
2,536,161.0,2008-08-02T18:49:07Z,502,9,<p>You can use ImageMagick's convert utility f...
3,538,156.0,2008-08-02T18:56:56Z,535,23,<p>One possibility is Hudson. It's written in...
4,541,157.0,2008-08-02T19:06:40Z,535,20,"<p>We run <a href=""http://buildbot.net/trac"">B..."


In [59]:
tags = pd.read_csv('data/Tags.csv', encoding='latin1')
tags.head()

,Id,Tag
0,469,python
1,469,osx
2,469,fonts
3,469,photoshop
4,502,python


In [60]:
# merge tags to questions 
tags['Tag'] = tags['Tag'].astype(str)
tag_series = tags.groupby('Id', as_index=True)['Tag'].apply(lambda x: ' '.join(x))
tag_series = pd.DataFrame(tag_series).reset_index()
q_shape = len(questions)
questions = pd.merge(questions, tag_series, how='left',  on='Id')
assert len(questions) == q_shape

In [70]:
questions.iloc[0].Body.replace()

"<p>I am using the Photoshop's javascript API to find the fonts in a given PSD.</p>\n\n<p>Given a font name returned by the API, I want to find the actual physical font file that that font name corresponds to on the disc.</p>\n\n<p>This is all happening in a python program running on OSX so I guess I'm looking for one of:</p>\n\n<ul>\n<li>Some Photoshop javascript</li>\n<li>A Python function</li>\n<li>An OSX API that I can call from python</li>\n</ul>\n"

In [94]:
import re 
import html
from bs4 import BeautifulSoup

re1 = re.compile(r'  +')

def soupify(x):
    return BeautifulSoup(x, "lxml").text

def cleanup(x):
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>','u_n').replace(' @.@ ','.').replace(
        ' @-@ ','-').replace('\\', ' \\ ').replace('\n', ' ').strip()
    return re1.sub(' ', html.unescape(x))

In [95]:
%%time
questions['Body'] = questions.Body.map(lambda x: soupify(x))
questions['Body'] = questions.Body.map(lambda x: cleanup(x))

CPU times: user 2min 58s, sys: 887 ms, total: 2min 58s
Wall time: 2min 58s


In [98]:
%%time
answers['Body'] = answers.Body.map(lambda x: soupify(x))
answers['Body'] = answers.Body.map(lambda x: cleanup(x))

CPU times: user 4min 30s, sys: 115 ms, total: 4min 30s
Wall time: 4min 30s


In [100]:
answers.head()

,Id,OwnerUserId,CreationDate,ParentId,Score,Body
0,497,50.0,2008-08-02T16:56:53Z,469,4,open up a terminal (Applications->Utilities->T...
1,518,153.0,2008-08-02T17:42:28Z,469,2,I haven't been able to find anything that does...
2,536,161.0,2008-08-02T18:49:07Z,502,9,You can use ImageMagick's convert utility for ...
3,538,156.0,2008-08-02T18:56:56Z,535,23,One possibility is Hudson. It's written in Jav...
4,541,157.0,2008-08-02T19:06:40Z,535,20,"We run Buildbot - Trac at work, I haven't used..."


In [ ]:

import numpy as np
from more_itertools import chunked
from math import ceil
from itertools import chain
from collections import Counter, defaultdict
from multiprocessing import Pool, cpu_count
import re
import os
import pickle
import fastText as ft

from keras.preprocessing.sequence import pad_sequences

import spacy
from spacy.symbols import ORTH


def flattenlist(listoflists):
    return list(chain.from_iterable(listoflists))

class Tokenizer():
    def __init__(self, lang='en'):
        self.tokenizer = spacy.load(lang)
        self.re_br = re.compile(r'<\s*br\s*/?>', re.IGNORECASE)
        for w in ('_eos_','_bos_','_unk_'):
            self.tokenizer.tokenizer.add_special_case(w, [{ORTH: w}])
            
    def sub_br(self, X): 
        return self.re_br.sub("\n", X)
    
    def spacy_tokenizer(self, X):
        return [x.text for x in self.tokenizer.tokenizer(self.sub_br(X.lower())) if not x.is_space]
    
    def proc_text(self, X):
        s = re.sub(r'([/#])', r' \1 ', X)
        s = re.sub(' {2,}', ' ', X)
        return self.spacy_tokenizer(s)
       
    @staticmethod
    def proc_all(X):
        tok = Tokenizer()
        return [tok.proc_text(x) for x in X]

    def fit_transform(self, X):
        core_usage = (cpu_count() + 1) // 2
        with Pool(core_usage) as p:
            chunk_size = ceil(len(X) / core_usage)
            results = p.map(Tokenizer.proc_all, chunked(X, chunk_size), chunksize=1)
        return flattenlist(results)
    
    def transform(self, X):
        return Tokenizer.proc_all(X)
    
    
    


class TextDataSet(object):
    def __init__(self, max_vocab, maxlen, min_freq=1, padding='pre'):
        self.max_vocab = max_vocab
        self.min_freq = min_freq
        self.maxlen = maxlen
        self.padding = padding
        self.tokenizer = Tokenizer()
    
    def fit(self, text, tokenize=True):
        if tokenize:
            text = self.tokenizer.fit_transform(text)
        self.freq = Counter(p for sent in text for p in sent)
        self.idx2word = [word for word, count in self.freq.most_common(self.max_vocab) if count > self.min_freq]
        self.idx2word.insert(0, '_unk_')
        self.idx2word.insert(1, '_pad_')
        self.idx2word.insert(2, '_bos_')
        self.idx2word.insert(3, '_eos_')
        self.word2idx = defaultdict(lambda: 0, {word: i for i, word in enumerate(self.idx2word)})
        self.pad_int = self.word2idx['_pad_']
        return text
        
    def fit_transform(self, text, tokenize=True):
        text = self.fit(text, tokenize=tokenize)
        text_padded = self.internal_transform(text, tokenize=False)
        return np.array(text_padded)
    
    def internal_transform(self, text, tokenize=True):
        if tokenize:       
            text = self.tokenizer.fit_transform(text)
        text_ints = np.array([[self.word2idx[i] for i in sent] for sent in text])
        text_padded = pad_sequences(text_ints, maxlen=self.maxlen, padding=self.padding, value=self.pad_int)
        return np.array(text_padded)
    
    def transform(self, text, tokenize=True, word2idx=None, maxlen=None, padding=None):
        if tokenize:       
            text = self.tokenizer.fit_transform(text)
        if word2idx:
            self.word2idx = word2idx
        if maxlen:
            self.maxlen = maxlen
        if padding:
            self.padding = padding
        text_ints = np.array([[self.word2idx[i] for i in sent] for sent in text])
        text_padded = pad_sequences(text_ints, maxlen=self.maxlen, padding=self.padding, value=self.pad_int)
        return np.array(text_padded)
    